# Rekurentne nevronske mreže

V prejšnjem modulu smo uporabljali bogate semantične reprezentacije besedila in preprost linearni klasifikator na vrhu vgrajenih predstavitev. Ta arhitektura zajame združeni pomen besed v stavku, vendar ne upošteva **vrstnega reda** besed, saj operacija združevanja na vrhu vgrajenih predstavitev odstrani to informacijo iz izvirnega besedila. Ker ti modeli ne morejo modelirati vrstnega reda besed, ne morejo reševati bolj zapletenih ali dvoumnih nalog, kot so generiranje besedila ali odgovarjanje na vprašanja.

Da bi zajeli pomen zaporedja besedila, moramo uporabiti drugo arhitekturo nevronske mreže, imenovano **rekurentna nevronska mreža** ali RNN. Pri RNN stavke pošiljamo skozi mrežo en simbol naenkrat, mreža pa ustvari neko **stanje**, ki ga nato skupaj z naslednjim simbolom ponovno pošljemo v mrežo.

Glede na vhodno zaporedje tokenov $X_0,\dots,X_n$, RNN ustvari zaporedje blokov nevronske mreže in to zaporedje trenira od začetka do konca z uporabo povratnega razširjanja napake. Vsak blok mreže sprejme par $(X_i,S_i)$ kot vhod in ustvari $S_{i+1}$ kot rezultat. Končno stanje $S_n$ ali izhod $X_n$ gre v linearni klasifikator, da ustvari rezultat. Vsi bloki mreže imajo enake uteži in se trenirajo od začetka do konca z enim prehodom povratnega razširjanja napake.

Ker se vektorska stanja $S_0,\dots,S_n$ prenašajo skozi mrežo, lahko ta model uči zaporedne odvisnosti med besedami. Na primer, ko se beseda *ne* pojavi nekje v zaporedju, se lahko nauči negirati določene elemente znotraj vektorskega stanja, kar vodi do negacije.

> Ker so uteži vseh blokov RNN na sliki enake, lahko isto sliko predstavimo kot en blok (na desni) z povratno zanko, ki prenaša izhodno stanje mreže nazaj na vhod.

Poglejmo, kako nam lahko rekurentne nevronske mreže pomagajo pri klasifikaciji našega nabora novic.


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## Preprost RNN klasifikator

V primeru preprostega RNN je vsaka rekurentna enota preprosto linearno omrežje, ki sprejme združen vhodni vektor in vektorsko stanje ter ustvari novo vektorsko stanje. PyTorch predstavlja to enoto z razredom `RNNCell`, omrežje takšnih celic pa kot plast `RNN`.

Za definiranje RNN klasifikatorja bomo najprej uporabili vgradno plast (embedding layer), da zmanjšamo dimenzionalnost vhodnega besedišča, nato pa bomo na vrhu dodali plast RNN:


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **Opomba:** Tukaj uporabljamo neizurjeno vgrajeno plast za enostavnost, vendar lahko za še boljše rezultate uporabimo vnaprej izurjeno vgrajeno plast z Word2Vec ali GloVe vgraditvami, kot je opisano v prejšnji enoti. Za boljše razumevanje lahko ta koda prilagodite tako, da deluje z vnaprej izurjenimi vgraditvami.

V našem primeru bomo uporabili podajalnik podatkov z izravnavo (padded data loader), tako da bo vsak paket vseboval več izravnanih zaporedij enake dolžine. RNN plast bo vzela zaporedje vgraditvenih tenzorjev in ustvarila dva izhoda:
* $x$ je zaporedje izhodov RNN celic na vsakem koraku
* $h$ je končno skrito stanje za zadnji element zaporedja

Nato uporabimo popolnoma povezani linearni klasifikator, da dobimo število razredov.

> **Opomba:** RNN-ji so precej zahtevni za učenje, saj je število plasti, vključenih v povratno propagacijo, precej veliko, ko so RNN celice razširjene vzdolž dolžine zaporedja. Zato moramo izbrati majhno hitrost učenja in mrežo učiti na večjem naboru podatkov, da dosežemo dobre rezultate. To lahko traja precej dolgo, zato je priporočljiva uporaba GPU.


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## Dolgoročni kratkoročni spomin (LSTM)

Ena glavnih težav klasičnih RNN-jev je tako imenovana težava **izginjajočih gradientov**. Ker se RNN-ji učijo od začetka do konca v enem prehodu z vzvratnim razširjanjem, imajo težave s prenašanjem napake do prvih slojev mreže, zaradi česar mreža ne more učiti odnosov med oddaljenimi tokeni. Eden od načinov za izogibanje tej težavi je uvedba **eksplicitnega upravljanja stanja** z uporabo tako imenovanih **vrat**. Dve najbolj znani arhitekturi te vrste sta: **Dolgoročni kratkoročni spomin** (LSTM) in **Enota z vrati za posredovanje** (GRU).

![Slika, ki prikazuje primer celice dolgoročnega kratkoročnega spomina](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

LSTM mreža je organizirana na način, podoben RNN-ju, vendar se iz sloja v sloj prenašata dve stanji: dejansko stanje $c$ in skriti vektor $h$. Pri vsaki enoti se skriti vektor $h_i$ združi z vhodom $x_i$, in skupaj nadzorujeta, kaj se zgodi s stanjem $c$ prek **vrat**. Vsaka vrata so nevronska mreža s sigmoidno aktivacijo (izhod v območju $[0,1]$), ki jih lahko razumemo kot bitno masko, ko jih pomnožimo z vektorskim stanjem. Obstajajo naslednja vrata (od leve proti desni na zgornji sliki):
* **vrata za pozabo** vzamejo skriti vektor in določijo, katere komponente vektorja $c$ moramo pozabiti in katere prenesti naprej.
* **vhodna vrata** vzamejo nekaj informacij iz vhoda in skritega vektorja ter jih vstavijo v stanje.
* **izhodna vrata** transformirajo stanje prek neke linearne plasti s $\tanh$ aktivacijo, nato pa izberejo nekatere njegove komponente z uporabo skritega vektorja $h_i$, da ustvarijo novo stanje $c_{i+1}$.

Komponente stanja $c$ lahko razumemo kot neke vrste zastavice, ki jih lahko vklopimo ali izklopimo. Na primer, ko v zaporedju naletimo na ime *Alice*, lahko predpostavimo, da se nanaša na ženski lik, in dvignemo zastavico v stanju, da imamo v stavku ženski samostalnik. Ko kasneje naletimo na frazo *in Tom*, bomo dvignili zastavico, da imamo množinski samostalnik. Tako lahko z manipulacijo stanja domnevno sledimo slovničnim lastnostim delov stavka.

> **Opomba**: Odličen vir za razumevanje notranje strukture LSTM je ta odlični članek [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) avtorja Christopherja Olaha.

Čeprav se notranja struktura LSTM celice morda zdi zapletena, PyTorch to implementacijo skriva znotraj razreda `LSTMCell` in ponuja objekt `LSTM` za predstavitev celotnega LSTM sloja. Tako bo implementacija LSTM klasifikatorja precej podobna preprostemu RNN-ju, ki smo ga videli zgoraj:


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## Zapakirane sekvence

V našem primeru smo morali vse sekvence v mini seriji zapolniti z ničelnimi vektorji. Čeprav to povzroči nekaj nepotrebne porabe pomnilnika, je pri RNN-jih še bolj kritično, da se ustvarijo dodatne RNN celice za zapolnjene vnose, ki sodelujejo pri učenju, vendar ne nosijo nobenih pomembnih vhodnih informacij. Veliko bolje bi bilo, če bi RNN trenirali le do dejanske dolžine sekvence.

Za to je v PyTorch uveden poseben format za shranjevanje zapolnjenih sekvenc. Predpostavimo, da imamo vhodno zapolnjeno mini serijo, ki izgleda takole:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
Tukaj 0 predstavlja zapolnjene vrednosti, dejanski vektor dolžin vhodnih sekvenc pa je `[5,3,1]`.

Da bi učinkovito trenirali RNN z zapolnjenimi sekvencami, želimo začeti treniranje prve skupine RNN celic z veliko mini serijo (`[1,6,9]`), nato pa zaključiti obdelavo tretje sekvence in nadaljevati treniranje s krajšimi mini serijami (`[2,7]`, `[3,8]`) in tako naprej. Tako je zapakirana sekvenca predstavljena kot en vektor - v našem primeru `[1,6,9,2,7,3,8,4,5]`, in vektor dolžin (`[5,3,1]`), iz katerega lahko enostavno rekonstruiramo izvirno zapolnjeno mini serijo.

Za ustvarjanje zapakirane sekvence lahko uporabimo funkcijo `torch.nn.utils.rnn.pack_padded_sequence`. Vse rekurentne plasti, vključno z RNN, LSTM in GRU, podpirajo zapakirane sekvence kot vhod in proizvajajo zapakiran izhod, ki ga lahko dekodiramo z uporabo `torch.nn.utils.rnn.pad_packed_sequence`.

Da bi lahko ustvarili zapakirano sekvenco, moramo omrežju posredovati vektor dolžin, zato potrebujemo drugačno funkcijo za pripravo mini serij:


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

Dejansko bo mreža zelo podobna `LSTMClassifier` zgoraj, vendar bo metoda `forward` prejela tako zapolnjen mini-sklop kot tudi vektor dolžin zaporedij. Po izračunu vdelave izračunamo zapakiran niz, ga posredujemo sloju LSTM in nato rezultat ponovno razpakiramo.

> **Opomba**: Pravzaprav razpakiranega rezultata `x` ne uporabljamo, saj v nadaljnjih izračunih uporabljamo izhod iz skritih slojev. Zato lahko razpakiranje v tej kodi popolnoma odstranimo. Razlog, da ga tukaj vključimo, je, da vam omogočimo enostavno spreminjanje te kode, če bi morali uporabiti izhod mreže v nadaljnjih izračunih.


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **Opomba:** Morda ste opazili parameter `use_pack_sequence`, ki ga posredujemo funkciji za učenje. Trenutno funkcija `pack_padded_sequence` zahteva, da je dolžinski zaporedni tenzor na napravi CPU, zato mora funkcija za učenje preprečiti prenos podatkov o dolžinskem zaporedju na GPU med učenjem. Implementacijo funkcije `train_emb` si lahko ogledate v datoteki [`torchnlp.py`](../../../../../lessons/5-NLP/16-RNN/torchnlp.py).


## Dvosmerne in večplastne RNN

V naših primerih so vse rekurzivne mreže delovale v eni smeri, od začetka zaporedja do konca. To se zdi naravno, saj spominja na način, kako beremo in poslušamo govor. Vendar pa, ker imamo v mnogih praktičnih primerih naključni dostop do vhodnega zaporedja, bi bilo smiselno izvajati rekurzivno računanje v obeh smereh. Takšne mreže imenujemo **dvosmerne** RNN, in jih lahko ustvarimo z dodajanjem parametra `bidirectional=True` konstruktorju RNN/LSTM/GRU.

Pri delu z dvosmerno mrežo potrebujemo dva vektorja skritega stanja, enega za vsako smer. PyTorch te vektorje kodira kot en vektor z dvakrat večjo velikostjo, kar je precej priročno, saj običajno posredujemo nastalo skrito stanje v popolnoma povezano linearno plast, pri čemer moramo le upoštevati to povečanje velikosti pri ustvarjanju plasti.

Rekurzivna mreža, enosmerna ali dvosmerna, zajame določene vzorce znotraj zaporedja in jih lahko shrani v vektor stanja ali prenese v izhod. Tako kot pri konvolucijskih mrežah lahko na prvo plast zgradimo drugo rekurzivno plast, da zajamemo vzorce višje ravni, ki so zgrajeni iz vzorcev nižje ravni, ki jih je izločila prva plast. To nas pripelje do pojma **večplastne RNN**, ki je sestavljena iz dveh ali več rekurzivnih mrež, kjer se izhod prejšnje plasti prenese v naslednjo plast kot vhod.

![Slika, ki prikazuje večplastno dolgotrajno-kratkoročno pomnilniško RNN](../../../../../translated_images/sl/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Slika iz [tega čudovitega prispevka](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) avtorja Fernanda Lópeza*

PyTorch omogoča enostavno konstrukcijo takšnih mrež, saj morate le dodati parameter `num_layers` konstruktorju RNN/LSTM/GRU, da samodejno zgradite več plasti rekurzije. To pa tudi pomeni, da se velikost skritega/stanja vektorja sorazmerno poveča, kar morate upoštevati pri obdelavi izhoda rekurzivnih plasti.


## RNN-ji za druge naloge

V tej enoti smo videli, da se RNN-ji lahko uporabljajo za klasifikacijo sekvenc, vendar dejansko lahko obravnavajo še veliko več nalog, kot so generiranje besedila, strojno prevajanje in druge. Te naloge bomo obravnavali v naslednji enoti.



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za strojno prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da upoštevate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo strokovno človeško prevajanje. Ne prevzemamo odgovornosti za morebitna nesporazumevanja ali napačne razlage, ki izhajajo iz uporabe tega prevoda.
